# Tutorial 3: Guage Freedom

# T3.3: Creating a center of orthogonality

In [1]:
using LinearAlgebra
include("ncon.jl");

[6151.6222667876345 4850.042960817324 6261.594348538992 5771.5778474892595 5617.054946183367 6040.048660891784 5698.080378485535 5509.522080081138 5421.511763597184 5875.032753233754; 5753.255501055685 4565.929821347633 5851.119968119611 5430.433634193764 5235.748890665384 5672.883492245647 5340.140674104736 5182.116649843504 5053.578931933436 5518.980904606542; 6042.980962248169 4754.654423489408 6116.205641459946 5647.561093328505 5485.643161183798 5902.570568822432 5563.543431994612 5368.527858492338 5278.840837427412 5770.305300236061; 5837.740264612892 4602.625349091237 5963.559749204849 5513.988408442315 5378.592196936004 5786.999519826078 5420.327622170717 5266.018216284652 5166.516580884291 5603.630854998331; 6037.757936324593 4751.822126715003 6108.296644408033 5664.795696633299 5503.2714965174955 5897.987010635677 5565.733561729086 5346.479108040387 5272.008614125013 5752.3221768786925; 6220.097078574446 4924.138437970154 6291.639610872725 5838.732312971178 5614.200573890645 

In [25]:
##### Ex.3.3(c): Creating a center of orthogonality by 'pulling through'
# define tensors
d = 3;
A = rand(d,d,d,d); B = rand(d,d,d);
C = rand(d,d,d); D = rand(d,d,d);
E = rand(d,d,d); F = rand(d,d,d);
G = rand(d,d,d);
# iterate QR decomps
DQ, DR = qr(reshape(D,d^2,d)); DQ = reshape(Matrix(DQ),d,d,d);
EQ, ER = qr(reshape(E,d^2,d)); EQ = reshape(Matrix(EQ),d,d,d);

Btilda = ncon(Any[B,DR,ER],Any[[1,2,-3],[-1,1],[-2,2]]) # I THINK THE INDICES ARE NAMED WRONGLY
# NOTE:Note that we use a convention such tensor indices are ordered from left-to-right along 
# the bottom then left-to-right along the top. 
#Btilda = ncon(Any[B,DR,ER],Any[[1,-3,2],[-1,1],[-2,2]]) 
BQ, BR = qr(reshape(Btilda,d^2,d)); BQ = reshape(Matrix(BQ),d,d,d);
FQ, FR = qr(reshape(F,d^2,d)); FQ = reshape(Matrix(FQ),d,d,d);
GQ, GR = qr(reshape(G,d^2,d)); GQ = reshape(Matrix(GQ),d,d,d);


Ctilda = ncon(Any[C,GR],Any[[1,-2,-3],[-1,1]]); # SAME ISSUE HERE ABOUT THE INDICES
CQ, CR = qr(reshape(Ctilda,d^2,d)); CQ = reshape(Matrix(CQ),d,d,d);

Aprime = ncon(Any[A,BR,FR,CR],Any[[1,-2,2,3],[-1,1],[-3,2],[-4,3]]);
# new network is formed from tensors: {Aprime,BQ,CQ,DQ,EQ,FQ,GQ}.

# check both networks evaluate to the same tensor
connectlist = Any[[3,-5,4,5],[1,2,3],[6,-10,5],[-1,-2,1],[-3,-4,2],[-6,-7,4],[-8,-9,6]];
H0 = ncon(Any[A,B,C,D,E,F,G],connectlist)[:];
H1 = ncon(Any[Aprime,BQ,CQ,DQ,EQ,FQ,GQ],connectlist)[:];
dH = norm(H0-H1)/norm(H0)

5.953712606226274e-16

# Setting a center of orthogonality method 2: 'Direct Orthogonalization'

Here we describe a method for setting a tensor $$\textbf{A}$$ within a network T as a center of orthogonality directly using a single eigen-decomposition for each branch.

In [53]:
##### Ex.3.3(c): Creating a center of orthogonality with 'direct orthogonalization'
# define tensors
d = 3;
A = rand(d,d,d,d); B = rand(d,d,d);
C = rand(d,d,d); D = rand(d,d,d);
E = rand(d,d,d); F = rand(d,d,d);
G = rand(d,d,d);

In [54]:
# compute density matrices and their principle square roots
rho1 = ncon(Any[B,D,E,B,D,E],Any[[5,6,-2],[1,2,5],[3,4,6],[7,8,-1],[1,2,7],[3,4,8]]);
rho2 = ncon(Any[F,F],Any[[1,2,-2],[1,2,-1]]);
rho3 = ncon(Any[C,G,C,G],Any[[3,5,-2],[1,2,3],[4,5,-1],[1,2,4]]);

In [55]:
rho1

3×3 Array{Float64,2}:
  96.1129  126.807  103.455
 126.807   171.935  139.971
 103.455   139.971  114.73

In [56]:
eigen(rho1)

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
values:
3-element Array{Float64,1}:
   0.4680259911687352
   1.9052180957271323
 380.4047237058798
vectors:
3×3 Array{Float64,2}:
  0.0243943   0.866317  0.498899
 -0.642946   -0.36855   0.671409
  0.765523   -0.337144  0.548005

In [57]:
d1, u1 = eigen(rho1); sq_d1 = sqrt.(abs.(d1));
d2, u2 = eigen(rho2); sq_d2 = sqrt.(abs.(d2));
d3, u3 = eigen(rho3); sq_d3 = sqrt.(abs.(d3));

In [58]:
d1

3-element Array{Float64,1}:
   0.4680259911687352
   1.9052180957271323
 380.4047237058798

In [59]:
abs.(d1)

3-element Array{Float64,1}:
   0.4680259911687352
   1.9052180957271323
 380.4047237058798

In [60]:
sqrt.(abs.(d1))

3-element Array{Float64,1}:
  0.6841242512648819
  1.3802963796689218
 19.503966871020875

In [62]:
X1 = u1*diagm(0 => sq_d1)*u1'; X1inv = u1*diagm(0 => (1 ./sq_d1))*u1';
X2 = u2*diagm(0 => sq_d2)*u2'; X2inv = u2*diagm(0 => (1 ./sq_d2))*u2';
X3 = u3*diagm(0 => sq_d3)*u3'; X3inv = u3*diagm(0 => (1 ./sq_d3))*u3';

In [64]:
# execute gauge changes
Aprime = ncon(Any[A,X1,X2,X3],Any[[1,-2,2,3],[-1,1],[-3,2],[-4,3]]);
Bprime = ncon(Any[B,X1inv],Any[[-1,-2,1],[1,-3]]);
Fprime = ncon(Any[F,X2inv],Any[[-1,-2,1],[1,-3]]);
Cprime = ncon(Any[C,X3inv],Any[[-1,-2,1],[1,-3]]);
# new network is formed from tensors: {Aprime,Bprime,Cprime,D,E,Fprime,G}

In [65]:
# check both networks evaluate to the same tensor
connectlist = Any[[3,-5,4,5],[1,2,3],[6,-10,5],[-1,-2,1],[-3,-4,2],[-6,-7,4],[-8,-9,6]];
H0 = ncon(Any[A,B,C,D,E,F,G],connectlist)[:];
H1 = ncon(Any[Aprime,Bprime,Cprime,D,E,Fprime,G],connectlist)[:];
dH = norm(H0-H1) / norm(H0)

8.166291274409351e-16

# Problem Set 3:

In [95]:
# First we define the tensor A according to the definition in the lecture 
d = 12 # dimension of an indices

A = zeros(d,d,d) # initializing the matrix A
for ni = 1:d
    for nj = 1:d
        for nk = 1:d
            A[ni,nj,nk]=sqrt(ni + 2*nj + 3*nk);
        end
    end
end
B = C = A; # 

In [96]:
# a). Contracting the network {A,B,C} to form the H tensor explicitly
H = ncon(Any[A,B,C], Any[[-1,-2,1], [1,-3,2],[2,-4,-5]]);
N = norm(H)

1.7331274432963647e7

In [97]:
# b). THIS CODE IS NOT CORRECT (CONCEPT OD OFTAINING L AND R)
# Although it is not completely clear to me but I think that this problem can
# be solved by first doing the SVD of matrix C, doing the necessary truncation
# and obtaining the C' matrix and then finally fo the LR decomposition of this 
# matrix. Let us go with it and then see if we did it correctly
F = svd(reshape(C,d, d^2));

chi = 2; # given rank where we are expected to do the truncation

C_prime = F.U[:,1:chi]*diagm(0 => F.S[1:chi])*F.Vt[1:chi,:];

C_prime = reshape(C_prime, d, d, d);

# now we want to do the QR decomposition
Q, R = qr(reshape(C_prime, d, d^2)); R = reshape(R, d, d, d);

# contracting the teensors again to obtain H1
H1 = ncon(Any[A,B,Q,R], Any[[-1,-2,1],[1,-3,2],[2,3],[3,-4,-5]]);

epsilon = norm(H-H1)/norm(H)

9.458269336625596e-7

In [108]:
# c)
# We want to calculate the tensor C' through pulling through approach
# We start with iterative QR decomposition
AQ, AR = qr(reshape(A, d^2, d)) ; A_prime = reshape(Matrix(AQ), d, d, d);


# Now we want to absorb AR in B
B_tilde = AR*reshape(B, d, d^2) ; B_tilde = reshape(B_tilde,d,d,d);

# Now we will fo the QR decomposition of B_prime
BQ, BR = qr(reshape(B_tilde, d^2, d)) ; B_prime = reshape(Matrix(BQ), d,d,d);

# Finally calculating C' be contracting BR into C
C_prime = reshape(BR*reshape(C, d, d^2),d,d,d);

In [99]:
H_prime = ncon(Any[A_prime,B_prime,C_prime], Any[[-1,-2,1], [1,-3,2],[2,-4,-5]]);


In [100]:
norm(H-H_prime)

2.0011836846420527e-8

In [18]:
size(AQ), size(BQ)

((144, 144), (144, 144))

In [19]:
size(AQ)



(144, 144)

In [20]:
size(A)

(12, 12, 12)

In [29]:
m = Matrix(AQ);
size(m)

(144, 12)

# Solutions from the tutorial

In [102]:
d = 12;
A = zeros(d,d,d);
for ni = 1:d
    for nj = 1:d
        for nk = 1:d
            A[ni,nj,nk] = sqrt(ni + 2*nj + 3*nk);
        end
    end
end
B = A;
C = A;

In [103]:
# (a) Evaluate Network
H = ncon(Any[A,B,C],Any[[-1,-2,1],[1,-3,2],[2,-4,-5]]);
nH = norm(H)

1.7331274432963647e7

In [104]:
# (b) truncated SVD
um,sm,vm = svd(reshape(C,d,d^2));
chi = 2;
CL = um[:,1:chi]*diagm(0 => sqrt.(sm[1:chi]));
CR = reshape(diagm(0 => sqrt.(sm[1:chi]))*(vm[:,1:chi]'),chi,d,d);
H1 = ncon(Any[A,B,CL,CR],Any[[-1,-2,1],[1,-3,2],[2,3],[3,-4,-5]]);
err1 = norm(H - H1) / nH

9.458269336703137e-7

In [105]:
# (c) create center of orthogonality at C
Qm, Rm = qr(reshape(A,d^2,d));
Ap = reshape(Matrix(Qm),d,d,d);
Qm, Rm = qr(reshape(ncon(Any[Rm,B],Any[[-1,1],[1,-2,-3]]),d^2,d));

# ONE CAN ALSO USE THE FOLLOWING MATRIX MULTIPLICATION INSTEAD OF NCON
# FUNCTION
#Bt = Rm*reshape(B, d, d^2); 
#Qm, Rm = qr(reshape(Bt, d^2, d)); 

Bp = reshape(Matrix(Qm),d,d,d);
Cp = ncon(Any[Rm,C],Any[[-1,1],[1,-2,-3]]);

#Cp = reshape(Rm*reshape(C, d, d^2), d,d,d);

# check result
Hp = ncon(Any[Ap,Bp,Cp],Any[[-1,-2,1],[1,-3,2],[2,-4,-5]]);
errp = norm(H - Hp) / nH # should be zero

In [ ]:
# (d) 
